<img style="float: left;;" src='Figures/iteso.jpg' width="100" height="200"/></a>

# <center> <font color= #000047> 5.- Feature Hashing </font> </center>

Feature Hashing, también conocido como el **"hashing trick"**, es una técnica para transformar variables categóricas de alta cardinalidad en un espacio de características de tamaño fijo. Es especialmente útil cuando el número de categorías posibles es muy grande o desconocido.

La **cardinalidad** de una variable categórica es el número de categorías únicas que contiene.

Se considera que una variable tiene **alta cardinalidad** cuando el número de categorías distintas es grande en relación al número de observaciones o al contexto del problema. No existe un umbral universal, pero algunas reglas prácticas son:

- Si el número de categorías es mayor a 10-20, puede considerarse moderadamente alta.
- Si hay decenas, cientos o miles de categorías únicas, es claramente alta cardinalidad.
- Si el número de categorías es comparable al número de filas (por ejemplo, un identificador único por registro), la cardinalidad es extremadamente alta.

**Ejemplos de variables de alta cardinalidad:**
- Códigos postales
- Nombres de productos
- IDs de usuario
- Direcciones de correo electrónico

**¿Por qué es importante?**
Las variables de alta cardinalidad pueden causar problemas de memoria y sobreajuste si se codifican con técnicas tradicionales como one-hot encoding. Por eso, técnicas como feature hashing son útiles en estos casos.

#### Ejemplo:

Supón que tienes 10,000 países distintos y decides usar m=100 buckets. Cada país se asigna a un bucket usando una función hash y la operación módulo. Así, solo necesitas 100 columnas para representar cualquier país, aunque pierdes la capacidad de distinguir perfectamente entre todos ellos si hay colisiones.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df=pd.read_csv('Países.csv')
df.head()

In [ ]:
# Cardinalidad de 'Países'


In [ ]:
#Suma acumulada


### ¿Cómo funciona Feature hashing?

1. **Definir el número de buckets (m):**
   Se elige un número fijo de columnas (buckets) donde se mapearán las categorías. Este número es mucho menor que la cantidad total de categorías posibles.

2. **Aplicar una función hash:**
   Cada categoría se transforma usando una función hash (por ejemplo, hash de Python, MurmurHash, etc.), que convierte el valor de la categoría en un número entero.

3. **Asignar a un bucket:**
   El valor hash se reduce al rango de buckets usando la operación módulo (`hash(categoría) % m`). Así, cada categoría se asigna a uno de los m buckets.

4. **Codificación:**
   Para cada observación, se crea un vector de longitud m, donde solo el bucket correspondiente a la categoría tiene un valor distinto de cero (por ejemplo, 1 o la frecuencia), y el resto son ceros. Si varias categorías caen en el mismo bucket, sus valores se suman (esto se llama colisión).

### Algunas Ventajas:
- Permite trabajar con variables categóricas de alta cardinalidad sin crear miles de columnas.
- El espacio de representación es fijo y controlable.
- Es eficiente en memoria y velocidad.

### Riegos que puede haber con es:
- Puede haber colisiones: diferentes categorías pueden terminar en el mismo bucket, perdiendo información.
- No es interpretable: no es posible saber a qué categoría corresponde cada bucket.


#### Ejemplo

In [ ]:
#Por conteo de categorías


#### Función hash

Una función hash es una función determinista que asigna un entero potencialmente ilimitado a un rango finito de enteros [1, m]. Dado que el dominio de entrada es potencialmente mayor que el rango de salida, varios números pueden asignarse al mismo resultado. Esto se llama una colisión. Una función hash uniforme asegura que aproximadamente la misma cantidad de números se asignen a cada uno de los m contenedores.

Visualmente, podemos pensar en una función hash como una máquina que recibe bolas numeradas (claves) y las dirige a uno de los m contenedores. Las bolas con el mismo número siempre serán dirigidas al mismo contenedor. Esto mantiene el espacio de características mientras reduce el almacenamiento y el tiempo de procesamiento durante el entrenamiento y evaluación de algún algoritmo de aprendizaje automático.

Las funciones hash pueden construirse para cualquier objeto que pueda representarse numéricamente (lo cual es cierto para cualquier dato que pueda almacenarse en una computadora): números, cadenas, estructuras complejas, etc.

In [ ]:
# Función Hash tomando en cuenta la frecuencia


In [ ]:
# sin conteo


In [ ]:
# Otra función hash


### Función hash simple (basado en el módulo)

### Usando librerías

#### FeatureHasher de scikit-learn

`FeatureHasher` es una clase de scikit-learn (`sklearn.feature_extraction.FeatureHasher`) que implementa la técnica de **feature hashing** (hashing trick) para transformar variables categóricas (o texto) en un espacio de características de tamaño fijo. Es especialmente útil para datos de alta cardinalidad y/o cuando el espacio de posibles categorías es muy grande o desconocido.

##### Funcionamiento

1. **Entrada esperada**:
   - Recibe una lista de diccionarios, una matriz dispersa o un DataFrame donde cada fila representa una muestra y cada columna una característica categórica o de texto.
   - Cada valor puede ser una cadena, un número o una lista de tokens (en el caso de texto).

2. **Proceso de hashing**:
   - Para cada par `(característica, valor)` en cada muestra, se calcula un hash usando una función hash rápida (por defecto, MurmurHash3 de 32 bits).
   - El hash se reduce al rango `[0, n_features-1]` usando la operación módulo, donde `n_features` es el número de columnas de salida (buckets).

3. **Asignación de valores**:
   - Si el valor es numérico, se suma ese valor en el bucket correspondiente.
   - Si el valor es categórico (cadena), se suma 1 en el bucket correspondiente.
   - Para evitar que los buckets sean siempre positivos (y reducir colisiones), el signo del valor se determina usando el hash (puede ser +1 o -1).

4. **Salida**:
   - Devuelve una matriz dispersa (`scipy.sparse`) de tamaño `(n_samples, n_features)`, donde cada fila es la representación hash de la muestra original.

##### argumentos de entrada principales

- `n_features`: Número de columnas (buckets) de salida. Controla la dimensionalidad y la probabilidad de colisiones.
- `input_type`: Tipo de entrada. Puede ser `'dict'`, `'pair'`, o `'string'`.
- `alternate_sign`: Si es `True` (por defecto), alterna el signo de los valores para reducir el sesgo por colisiones.

`FeatureHasher` de scikit-learn es una herramienta poderosa para convertir variables categóricas o texto en vectores numéricos de tamaño fijo, usando hashing eficiente y controlando la dimensionalidad, a costa de perder interpretabilidad y permitir colisiones controladas.

In [ ]:
## Ejemplo de uso
from sklearn.feature_extraction import FeatureHasher

# Supón que tienes una lista de diccionarios con variables categóricas
data = [
    {'color': 'red', 'shape': 'circle'},
    {'color': 'blue', 'shape': 'square'},
    {'color': 'green', 'shape': 'triangle'},
    {'color': 'red', 'shape': 'square'}
]
data_df = pd.DataFrame(data)
data_df

In [ ]:
# Crear el hasher con 8 buckets

# Transformar los datos

# X es una matriz dispersa (sparse matrix)



**MurmurHash3** es una función hash no criptográfica, rápida y eficiente, ampliamente utilizada para aplicaciones como hash tables y feature hashing. La versión de 32 bits produce un entero de 32 bits a partir de una cadena o secuencia de bytes.

- **No criptográfica:** No es segura para aplicaciones de seguridad, pero es muy rápida y tiene baja tasa de colisiones para uso general.
- **Determinística:** La misma entrada y semilla siempre producen el mismo hash.
- **Uniformidad:** Distribuye los valores de hash de manera uniforme, lo que es ideal para hashing en estructuras de datos y machine learning.

En scikit-learn y otras librerías, MurmurHash3 se usa internamente para convertir cadenas en índices de buckets de manera eficiente y reproducible.


In [ ]:
from sklearn.utils.murmurhash import murmurhash3_32

# Ejemplo: obtener el hash de una cadena
valor = "Mexico"
hash_val = murmurhash3_32(valor, seed=0)
print(hash_val)

In [ ]:
help(murmurhash3_32)

In [ ]:
# Función hash MurmurHash3 de 32 bits


In [ ]:
#!pip install category-encoders

#### Category Encoders

In [ ]:
#Usando category_encoders para dataset de países
